In [1]:
import os
import pandas as pd

# Get list of text files in the directory
text_files = [f for f in os.listdir('processed_data/text') if f.endswith('.txt')]

# Create lists to store data
names = []
texts = []

# Read each file and store its content
for file_name in text_files:
    with open(os.path.join('processed_data/text', file_name), 'r', encoding='utf-8') as file:
        names.append(file_name)
        texts.append(file.read())

# Create dataframe
df = pd.DataFrame({
    'Name': names,
    'text': texts
})

In [2]:
import re
from collections import defaultdict

def extract_api_number(text: str) -> str:
    if pd.isna(text):
        return {}
    
    # Pattern matches exactly 2 digits, dash, 3 digits, dash, 5 digits
    pattern = r'(\d{2})\s*-\s*(\d{3})\s*-\s*(\d{5})'
    
    # Find all matches in the text
    matches = re.findall(pattern, str(text))
    
    # Create dictionary with counts
    api_counts = defaultdict(int)
    for match in matches:
        api_number = '-'.join(match)  # Join tuple elements with dashes
        api_counts[api_number] += 1
        
    return dict(api_counts)

def extract_most_common_api_number(api_counts: dict) -> str:
    return max(api_counts.items(), key=lambda x: x[1])[0] if len(api_counts) > 0 else None

In [3]:
df['all_api_number'] = df['text'].apply(extract_api_number)
df['api_number'] = df['all_api_number'].apply(extract_most_common_api_number)

df['api_number'].describe()

count               77
unique              77
top       33-053-02102
freq                 1
Name: api_number, dtype: object

In [4]:
def extract_well_name(text: str) -> str:
    if pd.isna(text):
        return ''
    
    # Multiple patterns for Well Names
    patterns = [
        # Standard Well Name pattern
        r'(?:Well Name:|Well Name|WELL NAME:?)\s*([A-Za-z0-9\s\-&#]+)(?:\n|$)',
        # Well Name and Number pattern
        r'(?:Well Name and Number|WELL NAME AND NUMBER:?)\n([A-Za-z0-9\s\-&#]+)'
    ]
    
    # Create dictionary with counts
    well_counts = defaultdict(int)
    
    for pattern in patterns:
        matches = re.findall(pattern, str(text), re.IGNORECASE)
        for match in matches:
            # Clean up the well name by removing trailing numbers and special characters
            well_name = re.sub(r'\s+\d+-\d+[A-Z]?H?$', '', match.strip())
            if 'and number' in well_name.lower() or 'county & state' in well_name.lower():
                continue
            if well_name:  # Ignore empty matches
                well_counts[well_name] += 1
    
    return dict(well_counts)

def extract_most_common_well_name(well_counts: dict) -> str:
    if max(well_counts.values()) == 1:
        return max(well_counts.items(), key=lambda x: len(x[0]))[0] if well_counts else ''
    return max(well_counts.items(), key=lambda x: x[1])[0] if well_counts else ''


In [5]:
df['all_well_name'] = df['text'].apply(extract_well_name)
df['well_name'] = df['all_well_name'].apply(extract_most_common_api_number)
df['well_name'].describe()

count          77
unique         64
top       Atlanta
freq            6
Name: well_name, dtype: object

In [6]:
def extract_coordinates(text: str) -> dict:
    """
    Extracts coordinates from formats:
    - N48° 47' 06.83" | W103° 37' 36.52"
    - N48 6 44.8812 | W103 4 16.1923
    - Latitude: 48° 2' 49.420 N
    """
    if pd.isna(text):
        return {'latitude': {}, 'longitude': {}}
    
    patterns = [
        # Format with pipe separator
        r'[N](\d+)\s*[°\s]+(\d+)\s*[\'|\s]+(\d+\.?\d*)[\"|\s]*\|[^\S\r\n]*[W](\d+)\s*[°\s]+(\d+)\s*[\'|\s]+(\d+\.?\d*)',
        
        # Split format with degrees symbol
        r'(?:Latitude|LAT)[:\s]*(\d+)°\s*(\d+)\'\s*(\d+\.?\d*)\s*[N][\s\S]*?(?:Longitude|LONG)[:\s]*(\d+)°\s*(\d+)\'\s*(\d+\.?\d*)\s*[W]',
        
        # Format without symbols
        r'[N](\d+)\s+(\d+)\s+(\d+\.?\d*)\s*\|[^\S\r\n]*[W](\d+)\s+(\d+)\s+(\d+\.?\d*)',
        
        # Individual lat/long lines
        r'(?:Site\s+Centre\s+)?(?:Latitude|LAT)[:\s]*(\d+)°\s*(\d+)\'\s*(\d+\.?\d*)\s*[N]',
        r'(?:Longitude|LONG)[:\s]*(\d+)°\s*(\d+)\'\s*(\d+\.?\d*)\s*[W]'
    ]
    
    lat_counts = defaultdict(int)
    long_counts = defaultdict(int)
    
    def dms_to_decimal(degrees, minutes, seconds):
        try:
            return float(degrees) + float(minutes)/60 + float(seconds)/3600
        except (ValueError, TypeError):
            return None
    
    for pattern in patterns:
        matches = re.findall(pattern, str(text), re.IGNORECASE | re.MULTILINE)
        for match in matches:
            try:
                if len(match) == 6:  # Complete coordinate pair
                    lat_val = dms_to_decimal(match[0], match[1], match[2])
                    long_val = -dms_to_decimal(match[3], match[4], match[5])
                elif len(match) == 3:  # Individual coordinate
                    if 'latitude' in pattern.lower():
                        lat_val = dms_to_decimal(match[0], match[1], match[2])
                        continue
                    else:
                        long_val = -dms_to_decimal(match[0], match[1], match[2])
                        continue
                
                if lat_val is not None and long_val is not None:
                    if -90 <= lat_val <= 90 and -180 <= long_val <= 180:
                        lat_counts[f"{lat_val:.6f}"] += 1
                        long_counts[f"{long_val:.6f}"] += 1
            except (ValueError, TypeError, IndexError):
                continue
    
    return {
        'latitude': dict(lat_counts),
        'longitude': dict(long_counts)
    }

In [7]:
df['coordinates'] = df['text'].apply(extract_coordinates)
# df['coordinates'] = df['all_coordinates'].apply(extract_most_common_coordinates)
df['coordinates'].describe()

count                                    77
unique                                   58
top       {'latitude': {}, 'longitude': {}}
freq                                     20
Name: coordinates, dtype: object

In [ ]:
import mysql
import mysql.connector
from secrets_import import * 

def write_to_database(df, batch_size=1000):
    conn = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DB,
        # Add connection pooling and optimization parameters
        pool_size=5,
        pool_name="mypool",
        buffered=True
    )
    
    cursor = conn.cursor()
    
    try:
        values = []
        for index, row in df.iterrows():
            values.append((
                row['api_number'],
                row['well_name'],
                str(row['coordinates']['latitude']),
                str(row['coordinates']['longitude']),
                ''  # address
            ))
            
            
            if len(values) >= batch_size:
                cursor.executemany(
                    """INSERT INTO oil_wells_information 
                       (api_number, well_name, latitude, longitude, address) 
                       VALUES (%s, %s, %s, %s, %s)""",
                    values
                )
                conn.commit()
                values = []
        
        
        if values:
            cursor.executemany(
                """INSERT INTO oil_wells_information 
                   (api_number, well_name, latitude, longitude, address) 
                   VALUES (%s, %s, %s, %s, %s)""",
                values
            )
            conn.commit()
            
    except mysql.connector.Error as error:
        print(f"Failed to insert records into MySQL table: {error}")
        conn.rollback()
        
    finally:
        cursor.close()
        conn.close()


import time

start_time = time.time()
print(f"Starting database write for {len(df)} records...")

write_to_database(df)

end_time = time.time()
print(f"Database write completed in {end_time - start_time:.2f} seconds")


Starting database write for 77 records...
Database write completed in 21.45 seconds
